# Ensembl mysql

- https://www.ensembl.org/info/data/mysql.html
- https://www.ensembl.org/info/docs/api/core/core_schema.html

Install mysqlclient: https://pypi.org/project/mysqlclient/

In [1]:
import sqlmodel as sqm
import pandas as pd

In [2]:
def get_url(db="core"):
    return f"mysql+mysqldb://anonymous:@ensembldb.ensembl.org/homo_sapiens_{db}_108_38"

## core db

In [3]:
engine = sqm.create_engine(get_url(), future=False)

Pull the full `gene` table:

In [3]:
gene = pd.read_sql("SELECT * FROM gene", con=engine)
gene.shape

(70616, 16)

In [4]:
gene.head()

,gene_id,biotype,analysis_id,seq_region_id,seq_region_start,seq_region_end,seq_region_strand,display_xref_id,source,description,is_current,canonical_transcript_id,stable_id,version,created_date,modified_date
0,554,Mt_tRNA,1,132907,577,647,1,2873366.0,insdc,mitochondrially encoded tRNA-Phe (UUU/C) [Sour...,1,1587,ENSG00000210049,1,2006-05-12,2006-05-12 00:00:00
1,555,Mt_rRNA,1,132907,648,1601,1,2873345.0,insdc,mitochondrially encoded 12S rRNA [Source:HGNC ...,1,1588,ENSG00000211459,2,2006-07-03,2010-01-08 18:00:56
2,556,Mt_tRNA,1,132907,1602,1670,1,2873419.0,insdc,mitochondrially encoded tRNA-Val (GUN) [Source...,1,1589,ENSG00000210077,1,2006-05-12,2006-05-12 00:00:00
3,557,Mt_rRNA,1,132907,1671,3229,1,2873347.0,insdc,mitochondrially encoded 16S rRNA [Source:HGNC ...,1,1590,ENSG00000210082,2,2006-05-12,2010-01-08 18:00:56
4,558,Mt_tRNA,1,132907,3230,3304,1,2873391.0,insdc,mitochondrially encoded tRNA-Leu (UUA/G) 1 [So...,1,1591,ENSG00000209082,1,2006-05-12,2006-05-12 00:00:00


Pull the `xref` table:

In [5]:
xref = pd.read_sql("SELECT * FROM xref", con=engine)
xref.shape

(2330490, 8)

In [6]:
xref.head()

,xref_id,external_db_id,dbprimary_acc,display_label,version,description,info_type,info_text
0,1,2700,ENSG00000108821,ENSG00000108821,0,None,NONE,
1,2,50541,LRG_1,LRG_1,0,Locus Reference Genomic record for COL1A1,DIRECT,
2,3,50543,LRG_1t1,LRG_1t1,0,Locus Reference Genomic record for COL1A1,DIRECT,
3,4,2700,ENSG00000165370,ENSG00000165370,0,None,NONE,
4,5,50541,LRG_1000,LRG_1000,0,Locus Reference Genomic record for GPR101,DIRECT,


Pull the `external_db` table:

- EntrezGene: 1300
- HGNC: 1100
- MGI: 1400

In [22]:
external_db = pd.read_sql("SELECT * FROM external_db", con=engine)
external_db.shape

(478, 10)

In [20]:
external_db[external_db["type"] == "PRIMARY_DB_SYNONYM"].head()

,external_db_id,db_name,db_release,status,priority,db_display_name,type,secondary_db_name,secondary_db_table,description
16,821,FlyBaseName_gene,1,KNOWN,5,FlyBase gene name,PRIMARY_DB_SYNONYM,None,None,None
19,826,FlyBaseCGID_gene,1,XREF,5,FlyBase gene CGID,PRIMARY_DB_SYNONYM,None,None,None
29,1100,HGNC,1,KNOWNXREF,100,HGNC Symbol,PRIMARY_DB_SYNONYM,None,None,None
31,1300,EntrezGene,1,KNOWNXREF,250,NCBI gene (formerly Entrezgene),PRIMARY_DB_SYNONYM,None,None,None
32,1400,MGI,1,KNOWNXREF,270,MGI Symbol,PRIMARY_DB_SYNONYM,None,None,None


Pull the `external_synonym` table:

In [12]:
external_synonym = pd.read_sql("SELECT * FROM external_synonym", con=engine)
external_synonym.shape

(56200, 2)

In [13]:
external_synonym.head()

,xref_id,synonym
0,420056,A2MP
1,423501,ALOX12E
2,438805,CRYB2B
3,440287,bA144G6.6
4,440287,bA342C24.4


Pull `dobject_xref` genes

In [9]:
object_xref = pd.read_sql(
    "SELECT * FROM object_xref WHERE ensembl_object_type = 'Gene'", con=engine
)
object_xref.shape

(467563, 6)

In [8]:
object_xref.head()

,object_xref_id,ensembl_id,ensembl_object_type,xref_id,linkage_annotation,analysis_id
0,258269,554,Gene,315859,None,8381
1,627342,554,Gene,2873366,None,8381
2,258270,555,Gene,315863,None,8381
3,627336,555,Gene,2873345,None,8381
4,428421,555,Gene,399115,None,8381


## Queries

In [14]:
query_core = """
SELECT gene.stable_id, xref.display_label, gene.biotype, gene.description, external_synonym.synonym
FROM gene
LEFT JOIN xref ON gene.display_xref_id = xref.xref_id
LEFT JOIN external_synonym ON gene.display_xref_id = external_synonym.xref_id
"""

query_external = """
SELECT gene.stable_id, object_xref.xref_id, xref.dbprimary_acc, external_db.db_name
FROM gene
LEFT JOIN object_xref ON gene.gene_id = object_xref.ensembl_id
LEFT JOIN xref ON object_xref.xref_id = xref.xref_id
LEFT JOIN external_db ON xref.external_db_id = external_db.external_db_id
WHERE object_xref.ensembl_object_type = 'Gene' AND external_db.db_name IN ('HGNC', 'EntrezGene')
"""

Query for the basic gene annotations:

In [7]:
results_core = pd.read_sql(query_core, con=engine)
results_core.shape

(111512, 5)

In [9]:
results_core.head()

,stable_id,display_label,biotype,description,synonym
0,ENSG00000210049,MT-TF,Mt_tRNA,mitochondrially encoded tRNA-Phe (UUU/C) [Sour...,MTTF
1,ENSG00000210049,MT-TF,Mt_tRNA,mitochondrially encoded tRNA-Phe (UUU/C) [Sour...,trnF
2,ENSG00000211459,MT-RNR1,Mt_rRNA,mitochondrially encoded 12S rRNA [Source:HGNC ...,12S
3,ENSG00000211459,MT-RNR1,Mt_rRNA,mitochondrially encoded 12S rRNA [Source:HGNC ...,MOTS-c
4,ENSG00000211459,MT-RNR1,Mt_rRNA,mitochondrially encoded 12S rRNA [Source:HGNC ...,MTRNR1


In [12]:
len(results_core["stable_id"].unique())

70616

Query for external ids:

In [15]:
results_external = pd.read_sql(query_external, con=engine)
results_external.shape

(83469, 4)

In [16]:
results_external

,stable_id,xref_id,dbprimary_acc,db_name
0,ENSG00000198888,478987,4535,EntrezGene
1,ENSG00000198763,479001,4536,EntrezGene
2,ENSG00000198804,478802,4512,EntrezGene
3,ENSG00000210151,1027844,113219467,EntrezGene
4,ENSG00000198712,478817,4513,EntrezGene
...,...,...,...,...
83464,ENSG00000162543,2949466,HGNC:26354,HGNC
83465,ENSG00000134686,2898382,HGNC:3183,HGNC
83466,ENSG00000159023,2802492,HGNC:3377,HGNC
83467,ENSG00000198216,2757756,HGNC:1392,HGNC
